In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import random

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
# from sklearn.externals.joblib import parallel_backend
from sklearn.datasets import make_blobs
from sklearn.externals import joblib


import collections

from dask.distributed import Client, progress


import datetime

In [2]:
# Remove the outliers
def remove_outliers(file_path):
    # return the list of dataframes
    dataframe_lists = []
    # Creating the dataframe and selecting the required columns
    for i in range(64):
        df = pd.read_csv(file_path, usecols=[1,2,3,4], skiprows=i*72000, nrows = 72000, names=["lz","X","Y","Z"])
        df['radiusSquare'] = df['X']*df['X']+df['Y']*df['Y']+df['Z']*df['Z']
        df['radius'] = np.sqrt(df['radiusSquare']).round(1)
        df['freq'] = df.groupby(['lz','radius'])['radius'].transform('count')
        for j in range(64):
            maxfreq = df[(df['lz']==j) & (df['radius']!=0)]['freq'].max()
            while maxfreq>100:
                df.drop(df[(df['lz']==j) & (df['freq']==maxfreq)].index, inplace=True)
                maxfreq = df[(df['lz']==j) & (df['radius']!=0)]['freq'].max()
                df.drop(df[(df['lz']==j) & (df['radius']==0)].index, inplace=True)
        dataframe_lists.append(df.iloc[:,1:4])
    return dataframe_lists

In [3]:
# atm = remove_outliers("/home/kia/Collected-Datasets/DEBS2019/debs2019_initial_dataset/Atm/in.csv")

# This the second data set that contains multiple objects
allData= remove_outliers("/home/kia/Collected-Datasets/DEBS2019/debs2019_dataset2/in.csv")


mX=np.array(allData[0]['X'])
mY=np.array(allData[0]['Y'])
mZ=np.array(allData[0]['Z'])




In [4]:
DataForClustering =  np.array(list(zip(mX, mY, mZ))) 

clustering = DBSCAN(eps=1, min_samples=10).fit(DataForClustering)

labels = clustering.labels_


labels = labels + 1 
print(max(labels))
print(min(labels))

38
0


In [5]:
# clustering with DBSCAN 
def doClusteringWithDBSCAN(data):
    
    clustering = DBSCAN(eps=1, min_samples=16).fit(data)

    labels = clustering.labels_
    
    return max(labels)

In [6]:
# clustering with DBSCAN 
def doClusteringWithDBSCAN_parallel(data):
    
    client = Client(processes=False, threads_per_worker=4, n_workers=2, memory_limit='2GB')
    model = DBSCAN(eps=1, min_samples=16)
    
    with joblib.parallel_backend('dask'):
        model.fit(data)

    labels = clustering.labels_

    return max(labels)

In [7]:
# run a test. 
now = datetime.datetime.now()
maxIter=1
sum = 0 
for i in range(maxIter):
    data =  np.array(list(zip(np.array(allData[i]['X']), np.array(allData[i]['Y']), np.array(allData[i]['Z'])))) 
    numberOfClusters = doClusteringWithDBSCAN(data)
    


# print(sum)   
print(datetime.datetime.now() - now)    
    

0:00:00.181168


In [ ]:
now = datetime.datetime.now()
maxIter=2
sum = 0 
for i in range(maxIter):
    data =  np.array(list(zip(np.array(allData[i]['X']), np.array(allData[i]['Y']), np.array(allData[i]['Z'])))) 
    numberOfClusters = doClusteringWithDBSCAN_parallel(data)

# print(sum)   
print(datetime.datetime.now() - now)